## Importing necessary packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# preprocessing/decomposition
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, KernelPCA

# keras 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV

# model evaluation
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# supportive models
from sklearn.ensemble import GradientBoostingRegressor
# feature selection (from supportive model)
from sklearn.feature_selection import SelectFromModel

# to make results reproducible
seed = 42 # was 42

Using Theano backend.
Using cuDNN version 5105 on context None
Mapped name None to device cuda: TITAN X (Pascal) (0000:03:00.0)


In [2]:
# Read datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# save IDs for submission
id_test = test['ID'].copy()

In [4]:
# glue datasets together
total = pd.concat([train, test], axis=0)
print('initial shape: {}'.format(total.shape))

# binary indexes for train/test set split
is_train = ~total.y.isnull()

initial shape: (8418, 378)


In [5]:
# find all categorical features
cf = total.select_dtypes(include=['object']).columns
print total[cf].head()

   X0 X1  X2 X3 X4 X5 X6 X8
0   k  v  at  a  d  u  j  o
1   k  t  av  e  d  y  l  o
2  az  w   n  c  d  x  j  x
3  az  t   n  f  d  x  l  e
4  az  v   n  f  d  h  d  n


In [6]:
# make one-hot-encoding convenient way - pandas.get_dummies(df) function
dummies = pd.get_dummies(
    total[cf],
    drop_first=False # you can set it = True to ommit multicollinearity (crucial for linear models)
)

print('oh-encoded shape: {}'.format(dummies.shape))
print dummies.head()

oh-encoded shape: (8418, 211)
   X0_a  X0_aa  X0_ab  X0_ac  X0_ad  X0_ae  X0_af  X0_ag  X0_ai  X0_aj  ...   \
0     0      0      0      0      0      0      0      0      0      0  ...    
1     0      0      0      0      0      0      0      0      0      0  ...    
2     0      0      0      0      0      0      0      0      0      0  ...    
3     0      0      0      0      0      0      0      0      0      0  ...    
4     0      0      0      0      0      0      0      0      0      0  ...    

   X8_p  X8_q  X8_r  X8_s  X8_t  X8_u  X8_v  X8_w  X8_x  X8_y  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     1     0  
3     0     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     0  

[5 rows x 211 columns]


In [7]:
# get rid of old columns and append them encoded
total = pd.concat(
    [
        total.drop(cf, axis=1), # drop old
        dummies # append them one-hot-encoded
    ],
    axis=1 # column-wise
)

print('appended-encoded shape: {}'.format(total.shape))

appended-encoded shape: (8418, 581)


In [8]:
# recreate train/test again, now with dropped ID column
train, test = total[is_train].drop(['ID'], axis=1), total[~is_train].drop(['ID', 'y'], axis=1)

# drop redundant objects
del total

# check shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))


Train shape: (4209, 580)
Test shape: (4209, 579)


In [9]:
# Calculate additional features: dimensionality reduction components
n_comp=10 # was 10

# uncomment to scale data before applying decompositions
# however, all features are binary (in [0,1] interval), i don't know if it's worth something
train_scaled = train.drop('y', axis=1).copy()
test_scaled = test.copy()
'''
ss = StandardScaler()
ss.fit(train.drop('y', axis=1))

train_scaled = ss.transform(train.drop('y', axis=1).copy())
test_scaled = ss.transform(test.copy())
'''

# PCA
pca = PCA(n_components=n_comp, random_state=seed)
pca2_results_train = pca.fit_transform(train_scaled)
pca2_results_test = pca.transform(test_scaled)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train_scaled)
ica2_results_test = ica.transform(test_scaled)

# Factor Analysis
#fca = FactorAnalysis(n_components=n_comp, random_state=seed)
#fca2_results_train = fca.fit_transform(train_scaled)
#fca2_results_test = fca.transform(test_scaled)

# Append it to dataframes
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
    #train['fca_' + str(i)] = fca2_results_train[:,i-1]
    #test['fca_' + str(i)] = fca2_results_test[:, i-1]


print train.shape
print test.shape

(4209, 600)
(4209, 599)


/home/krohitm/anaconda2/envs/krohitm/lib/python2.7/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [11]:
# create augmentation by feature importances as additional features
t = train['y']
tr = train.drop(['y'], axis=1)

# Tree-based estimators can be used to compute feature importances
clf = GradientBoostingRegressor(
                max_depth=4, 
                learning_rate=0.005, 
                random_state=seed, 
                subsample=0.95, 
                n_estimators=200
)

# fit regressor
clf.fit(tr, t)

# df to hold feature importances
features = pd.DataFrame()
features['feature'] = tr.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

print features.tail()

# select best features
model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(tr)

test_reduced = model.transform(test.copy())

print train_reduced.shape
print test_reduced.shape

# dataset augmentation
train = pd.concat([train, pd.DataFrame(train_reduced)], axis=1)
test = pd.concat([test, pd.DataFrame(test_reduced)], axis=1)

# check new shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))

         importance
feature            
X119       0.041776
X118       0.044236
pca_6      0.091539
X315       0.128361
X314       0.610175
(4209, 17)
(4209, 17)

Train shape: (4209, 617)
Test shape: (4209, 616)


In [12]:
# define custom R2 metrics for Keras backend
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [12]:
from keras.optimizers import SGD#, Adagrad
# base model architecture definition
def create_model(learn_rate = 0.0001, momentum = 0.9):
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer=SGD(lr=learn_rate, momentum=momentum),
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    #print(model.summary())
    return model

In [13]:
# X, y preparation
X, y = train.drop('y', axis=1).values, train.y.values
print(X.shape)

# X_test preparation
X_test = test
print(X_test.shape)

# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X, 
    y, 
    test_size=0.25, 
    random_state=seed
)

(4209, 616)
(4209, 616)


In [14]:
# define path to save model
import os
model_path = 'keras_model_CV.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=10, # was 10
        verbose=1),
    
    ModelCheckpoint(
        model_path, 
        monitor='val_loss', 
        save_best_only=True, 
        verbose=0)
]

In [15]:
# initialize input dimension
input_dims = train.shape[1]-1

#activation functions for hidden layers
act_func = 'tanh' # could be 'relu', 'sigmoid', ...

# make np.seed fixed
np.random.seed(seed)

# initialize estimator, wrap model in KerasRegressor
#estimator = KerasRegressor(
#    build_fn=create_model, 
#    nb_epoch=100, 
#    batch_size=10,
#    verbose=1
#)

estimator = KerasRegressor(
    build_fn = create_model,
    verbose = 0)

In [ ]:
#grid search cv to get best hyperparameters viz. batch_size, epochs
#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 80, 100]
#learn_rate = [0.0001, 0.001, 0.01, 0.1]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

batch_size = [10, 50, 100]
epochs = [20, 40]
learn_rate = [0.0001, 0.001, 0.01]
momentum = [0.0, 0.4, 0.9]

param_grid = dict(batch_size=batch_size, nb_epoch=epochs, learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=5)
grid_result = grid.fit(X, y)

In [ ]:
#get results of grid search
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# define path to save model
import os
model_path = 'keras_model_CV.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=10, # was 10
        verbose=1),
    
    ModelCheckpoint(
        model_path, 
        monitor='val_loss', 
        save_best_only=True, 
        verbose=0)
]

In [ ]:
from sklearn.cross_validation import StratifiedKFold

def train_and_evaluate_model(model, X_tr, y_tr, X_val, y_val):
    model.fit(
    X_tr, 
    y_tr, 
    #nb_epoch=10, # increase it to 20-100 to get better results
    validation_data=(X_val, y_val),
    verbose=2,
    callbacks=callbacks,
    shuffle=True
    )

n_folds = 5
#data, labels, header_info = load_data()
skf = StratifiedKFold(y, n_folds=n_folds, shuffle=True, random_state=seed)

for i, (train, test) in enumerate(skf):
    print "Running Fold", i+1, "/", n_folds
    model = None # Clearing the NN.
    model = create_model()
    train_and_evaluate_model(model, X[train], y[train], X[test], y[test])

In [ ]:
# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

### Temporary check for results

In [ ]:
# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

In [ ]:
# predict results
res = estimator.predict(X_test.values).ravel()

# create df and convert it to csv
output = pd.DataFrame({'id': id_test, 'y': res})
output.to_csv('../results/test_check.csv', index=False)

In [ ]:
estimator.save_weights('mercedes_sgd_weights_x.h5')